In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import os
import zipfile
import shutil
from PIL import Image
from imgaug import augmenters as iaa
import numpy as np
import pandas as pd
import time
import dask.bag as db
import dask.diagnostics as dd
import random
from termcolor import colored
import cv2
import imutils

In [4]:
CWD = os.getcwd()
CWD

'/workspace/Brain-Tumor-Classification'

# Function

In [5]:
def load_images_from_folder(folder_path):
    images = []
    image_names = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        if os.path.isfile(img_path) and filename.endswith(('.png', '.jpg', '.jpeg')):
            img = Image.open(img_path).convert('L')
            images.append(np.array(img))
            image_names.append(filename)
    return np.array(images), image_names

In [6]:
def save_images(list_images, list_image_names, move_to, list_folder_path, prefix="aug"):
    move_to = "dataset/" + move_to
    move_path = os.path.join(CWD, move_to)

    if os.path.exists(move_path):
        shutil.rmtree(move_path)
        print(f"Removed existing directory: {move_path}")
    os.makedirs(move_path)

    for path, images, image_names in zip(list_folder_path, list_images, list_image_names):
        print(len(images))
        if os.path.exists(path):
            shutil.rmtree(path)
            print(f"Removed existing directory: {path}")
        os.makedirs(path)

        for i, img in enumerate(images):
                    if i < len(image_names):
                        base_name, ext = os.path.splitext(image_names[i])
                        augmented_image_name = f"{base_name}{ext}"
                    else:
                        augmented_image_name = f"{prefix}_{i}.png"

                    augmented_image_pil = Image.fromarray(img)
                    augmented_image_path = os.path.join(path, augmented_image_name)
                    augmented_image_pil.save(augmented_image_path)

In [7]:
def gamma_correction(images, gamma):
    images = images/255.0
    corrected_images = np.power(images, gamma)
    corrected_images = np.clip(corrected_images * 255.0, 0, 255).astype(np.uint8)

    return corrected_images

In [8]:
def crop_img(img):
	"""
	Finds the extreme points on the image and crops the rectangular out of them
	"""
	gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
	gray = cv2.GaussianBlur(gray, (3, 3), 0)

	# threshold the image, then perform a series of erosions +
	# dilations to remove any small regions of noise
	thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
	thresh = cv2.erode(thresh, None, iterations=2)
	thresh = cv2.dilate(thresh, None, iterations=2)

	# find contours in thresholded image, then grab the largest one
	cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
	cnts = imutils.grab_contours(cnts)
	c = max(cnts, key=cv2.contourArea)

	# find the extreme points
	extLeft = tuple(c[c[:, :, 0].argmin()][0])
	extRight = tuple(c[c[:, :, 0].argmax()][0])
	extTop = tuple(c[c[:, :, 1].argmin()][0])
	extBot = tuple(c[c[:, :, 1].argmax()][0])
	ADD_PIXELS = 0
	new_img = img[extTop[1]-ADD_PIXELS:extBot[1]+ADD_PIXELS, extLeft[0]-ADD_PIXELS:extRight[0]+ADD_PIXELS].copy()
	
	return new_img

# Implementation

In [9]:
# folder_paths = [
#     os.path.join(cwd, 'dataset', 'raw', 'Training', 'glioma'),
#     os.path.join(cwd, 'dataset', 'raw', 'Training', 'meningioma'),
#     os.path.join(cwd, 'dataset', 'raw', 'Training', 'pituitary'),
#     os.path.join(cwd, 'dataset', 'raw', 'Training', 'notumor')
# ]

# GLIOMA_PATH = RAW_FOLDER[0]
# MENINGIOMA_PATH = RAW_FOLDER[1]
# PITUITARY_PATH = RAW_FOLDER[2]
# NORMAL_PATH = RAW_FOLDER[3]

In [10]:
# glioma, glioma_names = load_images_from_folder(GLIOMA_PATH)
# meningioma, meningioma_names = load_images_from_folder(MENINGIOMA_PATH)
# pituitary, pituitary_names = load_images_from_folder(PITUITARY_PATH)
# normal, normal_names = load_images_from_folder(NORMAL_PATH)

# print(len(glioma))
# print(len(meningioma))
# print(len(pituitary))
# print(len(normal))

In [11]:
# glioma = add_noise(glioma, 0, 5)
# glioma = gamma_correction(glioma, 1.2)

In [12]:
# meningioma = add_noise(meningioma, 0, 5)
# meningioma = gamma_correction(meningioma, 1.2)

In [13]:
# pituitary = add_noise(pituitary, 0, 5)
# pituitary = gamma_correction(pituitary, 1.2)

In [14]:
# normal = add_noise(normal, 0, 5)
# normal = gamma_correction(normal, 1.2)

In [15]:
# PROCESSED_PATH = [
#     CWD + '/dataset/processed/glioma_tumor',
#     CWD + '/dataset/processed/meningioma_tumor',
#     CWD + '/dataset/processed/pituitary_tumor',
#     CWD + '/dataset/processed/normal'
# ]

# list_images = [glioma, meningioma, pituitary, normal]
# list_image_names = [glioma_names, meningioma_names, pituitary_names, normal_names]

# save_images(list_images, list_image_names, 'processed', PROCESSED_PATH, prefix='aug')

In [11]:
training = "dataset/raw/Training"
testing = "dataset/raw/Testing"
TRAINING_DIR = os.listdir(training)
TESTING_DIR = os.listdir(testing)
IMG_SIZE = 256

for dir in TRAINING_DIR:
	save_path = 'dataset/processed/Training/'+ dir
	path = os.path.join(training,dir)
	image_dir = os.listdir(path)
	for img in image_dir:
		image = cv2.imread(os.path.join(path,img))
		new_img = crop_img(image)
		new_img = cv2.resize(new_img,(IMG_SIZE,IMG_SIZE))
		if not os.path.exists(save_path):
			os.makedirs(save_path)
		cv2.imwrite(save_path+'/'+img, new_img)
	
for dir in TESTING_DIR:
	save_path = 'dataset/processed/Testing/'+ dir
	path = os.path.join(testing,dir)
	image_dir = os.listdir(path)
	for img in image_dir:
		image = cv2.imread(os.path.join(path,img))
		new_img = crop_img(image)
		new_img = cv2.resize(new_img,(IMG_SIZE,IMG_SIZE))
		if not os.path.exists(save_path):
			os.makedirs(save_path)
		cv2.imwrite(save_path+'/'+img, new_img)

## B0 = 224

In [16]:
training = "dataset/raw/Training"
testing = "dataset/raw/Testing"
TRAINING_DIR = os.listdir(training)
TESTING_DIR = os.listdir(testing)
IMG_SIZE = 224

for dir in TRAINING_DIR:
	save_path = 'dataset/train_224/Training/'+ dir
	path = os.path.join(training,dir)
	image_dir = os.listdir(path)
	for img in image_dir:
		image = cv2.imread(os.path.join(path,img))
		new_img = crop_img(image)
		new_img = cv2.resize(new_img,(IMG_SIZE,IMG_SIZE))
		if not os.path.exists(save_path):
			os.makedirs(save_path)
		cv2.imwrite(save_path+'/'+img, new_img)
	
for dir in TESTING_DIR:
	save_path = 'dataset/train_224/Testing/'+ dir
	path = os.path.join(testing,dir)
	image_dir = os.listdir(path)
	for img in image_dir:
		image = cv2.imread(os.path.join(path,img))
		new_img = crop_img(image)
		new_img = cv2.resize(new_img,(IMG_SIZE,IMG_SIZE))
		if not os.path.exists(save_path):
			os.makedirs(save_path)
		cv2.imwrite(save_path+'/'+img, new_img)

In [20]:
training = "dataset/raw/Training"
testing = "dataset/raw/Testing"
TRAINING_DIR = os.listdir(training)
TESTING_DIR = os.listdir(testing)
IMG_SIZE = 300

for dir in TRAINING_DIR:
	save_path = 'dataset/train_300/Training/'+ dir
	path = os.path.join(training,dir)
	image_dir = os.listdir(path)
	for img in image_dir:
		image = cv2.imread(os.path.join(path,img))
		new_img = crop_img(image)
		new_img = cv2.resize(new_img,(IMG_SIZE,IMG_SIZE))
		if not os.path.exists(save_path):
			os.makedirs(save_path)
		cv2.imwrite(save_path+'/'+img, new_img)
	
for dir in TESTING_DIR:
	save_path = 'dataset/train_300/Testing/'+ dir
	path = os.path.join(testing,dir)
	image_dir = os.listdir(path)
	for img in image_dir:
		image = cv2.imread(os.path.join(path,img))
		new_img = crop_img(image)
		new_img = cv2.resize(new_img,(IMG_SIZE,IMG_SIZE))
		if not os.path.exists(save_path):
			os.makedirs(save_path)
		cv2.imwrite(save_path+'/'+img, new_img)

In [21]:
training = "dataset/raw/Training"
testing = "dataset/raw/Testing"
TRAINING_DIR = os.listdir(training)
TESTING_DIR = os.listdir(testing)
IMG_SIZE = 480

for dir in TRAINING_DIR:
	save_path = 'dataset/train_480/Training/'+ dir
	path = os.path.join(training,dir)
	image_dir = os.listdir(path)
	for img in image_dir:
		image = cv2.imread(os.path.join(path,img))
		new_img = crop_img(image)
		new_img = cv2.resize(new_img,(IMG_SIZE,IMG_SIZE))
		if not os.path.exists(save_path):
			os.makedirs(save_path)
		cv2.imwrite(save_path+'/'+img, new_img)
	
for dir in TESTING_DIR:
	save_path = 'dataset/train_480/Testing/'+ dir
	path = os.path.join(testing,dir)
	image_dir = os.listdir(path)
	for img in image_dir:
		image = cv2.imread(os.path.join(path,img))
		new_img = crop_img(image)
		new_img = cv2.resize(new_img,(IMG_SIZE,IMG_SIZE))
		if not os.path.exists(save_path):
			os.makedirs(save_path)
		cv2.imwrite(save_path+'/'+img, new_img)